In [3]:
from pyspark.sql import SparkSession
from pyspark import  SparkContext
import os
import pandas as pd

In [4]:
spark = SparkSession.builder.master("yarn").getOrCreate()

/spark/bin/load-spark-env.sh: line 68: ps: command not found
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/03/14 06:12:39 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/03/14 06:12:40 WARN Client: Neither spark.yarn.jars nor spark.yarn.archive is set, falling back to uploading libraries under SPARK_HOME.


In [5]:
import pandas as pd

# Sample data for the Student table
student_data = {
    'name': ['Jane', 'Pascal', 'Xi', 'Jack'],
    'continent': ['America', 'Europe', 'Asia', 'America']
}

# Creating a pandas DataFrame for the Student table
student_df = pd.DataFrame(student_data)

# Pivot the continent column
pivot_df = student_df.pivot(columns='continent', values='name')

# Reorder the columns
pivot_df = pivot_df[['America', 'Asia', 'Europe']]

# Sort the values alphabetically within each column
pivot_df = pivot_df.apply(lambda x: sorted(x, key=lambda v: str(v).lower()))

# Display the result
print(student_df)
print()
print(pivot_df)




df_person = spark.createDataFrame(student_df)
df_person.createOrReplaceTempView("Student")



     name continent
0    Jane   America
1  Pascal    Europe
2      Xi      Asia
3    Jack   America

continent America Asia  Europe
0            Jack  NaN     NaN
1            Jane  NaN     NaN
2             NaN  NaN     NaN
3             NaN   Xi  Pascal


In [6]:
query = """
    select
        *,
        row_number() over (partition by continent order by name asc) as rk
    from Student s
"""

In [13]:
query_1 = """
    select * from (
        select
            *,
            row_number() over (partition by continent order by name asc) as rk
        from Student s
    )
    pivot (
        max(name)
        for continent in (
            'America', 'Asia', 'Europe'
        )
    )

"""

In [11]:
spark.sql(query).show()

+------+---------+---+
|  name|continent| rk|
+------+---------+---+
|  Jack|  America|  1|
|  Jane|  America|  2|
|    Xi|     Asia|  1|
|Pascal|   Europe|  1|
+------+---------+---+



In [14]:
spark.sql(query_1).show()

+---+-------+----+------+
| rk|America|Asia|Europe|
+---+-------+----+------+
|  1|   Jack|  Xi|Pascal|
|  2|   Jane|NULL|  NULL|
+---+-------+----+------+

